In [1]:
from konlpy.corpus import kobill
from collections import Counter
from struct import pack, unpack

C = list()
V = list()
TDM = dict()

open('posting.dat', 'wb').close()

for file in kobill.fileids():
    C.append({'path':file,'maxfreq':0, 'length':0})

for d in C:
    i = C.index(d)
    localTDM = Counter(kobill.open(d['path']).read().split())
    C[i]['maxfreq'] = max(localTDM.values())
    
    for t, f in localTDM.items():
        if t not in V:
            V.append(t)
        j = V.index(t)
        
        if j in TDM.keys():
            TDM[j]['docfreq'] += 1
            info = (i,f,TDM[j]['pos'])
        else:
            TDM[j] = {'docfreq':1, 'pos':-1}
            info = (i,f,-1)

        fp = open('posting.dat', 'ab')
        pos = fp.tell()
        fp.write(pack('iii', *info))
        fp.close()

        TDM[j] = pos

TypeError: 'int' object is not subscriptable

In [2]:
from math import log

tf1 = lambda freq:1 if freq > 0 else 0 # binary
tf2 = lambda freq:freq # raw count
tf3 = lambda freq, sumfreq: freq/sumfreq # term freq
tf4 = lambda freq: log(1+freq) # log normalization
tf6 = lambda k, freq, maxfreq: k + (1-k)*(freq/maxfreq) # double normalization K
tf5 = lambda freq, maxfreq: tf6(.5, freq, maxfreq) # double normalization .5

idf1 = lambda:1
idf2 = lambda df, N:log(N/df) # nt =|tㅌd, dㅌC| => df
idf3 = lambda df, N:log(N/(1+df))+1
idf4 = lambda df, maxdf:log(maxdf/(1+df))
idf5 = lambda df, N:log((N-df)/df)

In [4]:
WDM = dict()

open('weights.dat', 'wb').close()

for t, tinfo in TDM.items():
    pos = tinfo['pos']
    df = tinfo['docfreq']
    
    with open('posting.dat', 'rb') as fp:
        fp2 = open('weights.dat', 'ab')
        wpos = fp2.tell()
        
        while pos != -1:
            fp.seek(pos)
            info = unpack('iii', fp.read(12))
            pos = info[-1]
            
            w = tf5(info[1], C[info[0]]['maxfreq'])*\
            idf2(df, len(C))
            winfo = (info[0], w)
            fp2.write(pack('if', *winfo))
        fp2.close()

        WDM[t] = {'pos':wpos,'df':df}

TypeError: 'int' object is not subscriptable

In [ ]:
Dist, Angle

In [11]:
# Distance
result = dict()

Q = kobill.open(C[0]['path']).read()
Q_tokenize = Counter(Q.split())
Q_maxfreq = max(Q_tokenize.values())
QWM = dict()

for t,f in Q_tokenize.items():
    w = tf5(f, Q_maxfreq)*idf2(TDM[V.index(t)]['docfreq'], len(C))
    QWM[V.index(t)] = w
    
for j in range(len(V)):
    w1 = QWM[j] if j in QWM.keys() else 0
    for info in WDM[j]:
        with open('weights.dat', 'rb') as fp:
            for pos in range(info['df']):
                fp.seek(info['pos']+(8*pos))
                w2 = unpack('if', fp.read(8))
                
                if w2[0] not in result.keys():
                    result[w2[0]] = (w1 - w2[1])**2
                else:
                    result[w2[0]] += (w1 - w2[1])**2
                    
result = {i:sqrt(ws) for i, ws in result.items()}
sorted(result.items(), key=lambda r: r[-1], reverse=False)

TypeError: 'int' object is not subscriptable

In [6]:
from math import sqrt

In [7]:
from nltk.tokenize import sent_tokenize
Q = sent_tokenize(kobill.open(C[2]['path']).read())[5]
C[2]['path']

'1809892.txt'

In [8]:
len(kobill.open(C[5]['path']).read().split()),\
len(set(kobill.open(C[5]['path']).read().split()))

(394, 256)

In [9]:
len(kobill.open(C[2]['path']).read().split()),\
len(set(kobill.open(C[2]['path']).read().split()))

(984, 611)

In [10]:
len(kobill.open(C[0]['path']).read().split()),\
len(set(kobill.open(C[0]['path']).read().split()))

(841, 541)

In [12]:
from os import listdir

def fileids(path):
    fileList = list()
    
    path = path + ('' if path[-1] == '/' else '/')
    
    for f in listdir(path):
        if f.endswith('.txt'):
            fileList.append(path+f)
    
    return fileList

In [14]:
C = [{'path':f, 'maxfreq':0, 'length':0} for f in fileids('news')]

In [ ]:
import re

p1 = re.compile('[a-zA-Z0-9_-]+@(?:[.]?[a-zA-Z0-9_-]+)+')
p2 = re.compile('[{}]'.format(re.escape('''"'='"''')))
p3 = re.compile('[\(\[].+?[\)\]]')
p4 = re.compile('[a-zA-Z]'+)
p5 = re.compile('\s+')
p6 = re.compile('[^ㄱ-하-|가-힣0-9.]')
p7 = re.compile('\s.[.]')

def preprocessing1(d, *patterns):
    for i in range(len(patterns)):
        d = patterns[i].sub(' ', d)
    return d.strip()

In [ ]:
def Indexer(file):
    with open(file, 'r', encoding='utf8') as fp:
        d = fp.read()
        print(d)
        print('========================')
        print(preprocessing1(d, p1, p2, p3))